In [5]:
#!pip install cplex
#Ipip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
#import pandas as pd

# Create model
mdl = ConcreteModel()

# Define Sets
mdl.N1 = Set(initialize=range(1, 5))
mdl.N2 = Set(initialize=[i for i in range(1, 6) if i != 3])
mdl.N3 = Set(initialize=[i for i in range(1, 6) if i != 1 and i != 4])
mdl.machines = Set(initialize=[1, 2, 3, 4, 5])
mdl.parts = Set(initialize=['a', 'b', 'c'])

# Define Variables
mdl.A = Var(mdl.N1, within=NonNegativeIntegers)
mdl.B = Var(mdl.N2, within=NonNegativeIntegers)
mdl.C = Var(mdl.N3, within=NonNegativeIntegers)
mdl.Y = Var(mdl.parts,mdl.parts,mdl.machines,within = Binary)
mdl.U = Var(within =NonNegativeIntegers)


T_df = pd.DataFrame([[10,8,0],[6,9,9],[12,0,14],[22,19,0],[0,7,6]],columns = list(mdl.parts),index = list(mdl.machines))
T_dict = {(i,j):T_df.loc[(i,j)] for i in T_df.index for j in T_df.columns}
mdl.T = Param(mdl.machines,mdl.parts,initialize = T_dict)


# Define binary variables
mdl.m1 = Var(within=Binary)
mdl.m2 = Var(within=Binary)
mdl.m2_prime = Var(within=Binary)
mdl.m2_double_prime = Var(within=Binary)
mdl.m3 = Var(within=Binary)  # Define m3
mdl.m4 = Var(within=Binary)  # Define m4

# Define Objective Function
mdl.obj = Objective(expr=mdl.U, sense=minimize)

# Define Constraints
M = 10000  # Large constant
#mdl.consList = Constraintlist()


mdl.obj_cons1 = Constraint(expr=mdl.U >= mdl.A[4] + mdl.T[4, 'a'])#object cons
mdl.obj_cons2 = Constraint(expr=mdl.U >= mdl.B[5] + mdl.T[5, 'b'])#object cons
mdl.obj_cons3 = Constraint(expr=mdl.U >= mdl.C[5] + mdl.T[5, 'c'])#object cons


# 4 min rest for the machine1
mdl.cons1 = Constraint(expr = mdl.A[1] + mdl.T[1,"a"] + 4 <= mdl.B[1]+M*(1-mdl.Y["a","b",1]))
mdl.cons2 = Constraint(expr = mdl.B[1] + mdl.T[1,"b"] + 4 <= mdl.A[1] + M*mdl.Y["a","b",1])


# 10 min rest for the machine 2
mdl.cons3 = Constraint(expr=mdl.A[2] + mdl.T[2, 'a'] + 10 <= mdl.B[2] + M*(1-mdl.Y["a","b",2]))
mdl.cons4 = Constraint(expr=mdl.B[2] + mdl.T[2, 'b'] + 10 <= mdl.A[2] + M*mdl.Y["a","b",2])

mdl.cons5 = Constraint(expr=mdl.A[2] + mdl.T[2, 'a'] + 10 <= mdl.C[2] + M*(1-mdl.Y["a","c",2]))
mdl.cons6 = Constraint(expr=mdl.C[2] + mdl.T[2, 'c'] + 10 <= mdl.A[2] + M*mdl.Y["a","c",2])

mdl.cons7 = Constraint(expr=mdl.B[2] + mdl.T[2, 'b'] + 10 <= mdl.C[2] + M*(1-mdl.Y["b","c",2]))
mdl.cons8 = Constraint(expr=mdl.C[2] + mdl.T[2, 'c'] + 10 <= mdl.B[2] + M*mdl.Y["b","c",2])

# 5 min rest for the machine 3

mdl.cons9 = Constraint(expr=mdl.A[3] + mdl.T[3, 'a'] + 5 <= mdl.C[3] + M*(1-mdl.Y["a","c",3]))
mdl.cons10 = Constraint(expr=mdl.C[3] + mdl.T[3, 'c'] + 5 <= mdl.A[3] + M*mdl.Y["a","c",3])


# 20 min rest for the machine 4

mdl.cons11 = Constraint(expr=mdl.A[4] + mdl.T[4, 'a'] + 20 <= mdl.B[4] + M*(1-mdl.Y["a","b",4]))
mdl.cons12 = Constraint(expr=mdl.B[4] + mdl.T[4, 'b'] + 20 <= mdl.A[4] + M*(mdl.Y["a","b",4]))

# cons for the manchine 5 
mdl.cons13 = Constraint(expr=mdl.B[5] + mdl.T[5, 'b'] <= mdl.C[5] + M*(1-mdl.Y["b","c",5]))
mdl.cons14 = Constraint(expr=mdl.C[5] + mdl.T[5, 'c'] <= mdl.B[5] + M*mdl.Y["b","c",5])


# 15 lag for partB finish time against partC finish time:
mdl.cons15 = Constraint(expr=mdl.C[5] - mdl.T[5, 'c'] + 15 <= mdl.B[5] + mdl.T[5, "b"])

# partA finish deadline:
mdl.cons16 = Constraint(expr=mdl.A[4] + mdl.T[4 , "a"]<= 60)

# machines order cons:
cons = 17
for i in mdl.N1:
    for j in mdl.N1:
        if(j>i):
            mdl.add_component("cons"+str(cons),Constraint(expr = mdl.A[i] + mdl.T[i,"a"]<=mdl.A[j]))
            cons+=1
print(cons)
for i in mdl.N2:
    for j in mdl.N2:
        if(j>i):
            mdl.add_component("cons"+str(cons),Constraint(expr = mdl.B[i] + mdl.T[i,"b"]<=mdl.B[j]))
            cons+=1
print(cons)
for i in mdl.N3:
    for j in mdl.N3:
        if(j>i):
            mdl.add_component("cons"+str(cons),Constraint(expr = mdl.C[i] + mdl.T[i,"c"]<=mdl.C[j]))
            cons+=1
print(cons)
        
        
# Solve the model
solver = SolverFactory('cplex_direct')
solver.solve(mdl)

# Display the results
print("Objective Value (U) =", value(mdl.U))
print("Starting times of machines on section A:")
for i in mdl.N1:
    print("Machine A", i, ": Start Time =", value(mdl.A[i]))
print("Starting times of machines on section B:")
for i in mdl.N2:
    print("Machine B", i, ": Start Time =", value(mdl.B[i]))
print("Starting times of machines on section C:")
for i in mdl.N3:
    print("Machine C", i, ": Start Time =", value(mdl.C[i]))




NameError: name 'pd' is not defined